# Регулярные выражения

<img width='450px' src="images/use-regex.jpg"></img>

Некоторые люди, когда сталкиваются с проблемой, думают:

    «Я знаю, я решу её с помощью регулярных выражений.»

Теперь у них **ДВЕ** проблемы.

## Что почитать?

1. Хороший [«быстрый старт»](https://tproger.ru/translations/regular-expression-python/) с задачками от TProger.
2. Отличный [туториал](https://habr.com/post/349860/) с задачками на Хабре.
3. [Документация](https://docs.python.org/3/library/re.html) к стандартной библиотеке.

In [93]:
import re

Отладчик регулярных выражений: https://www.debuggex.com/

# Структура URL

http://www.example.com:80/path/to/myfile.html?key1=value1&key2=value2#SomewhereInTheDocument

<img width='480px' src="https://mdn.mozillademos.org/files/8013/mdn-url-protocol@x2.png"></img>
<img width='480px' src="https://mdn.mozillademos.org/files/8015/mdn-url-protocol@x2.png"></img>
<img width='480px' src="https://mdn.mozillademos.org/files/8017/mdn-url-protocol@x2.png"></img>
<img width='480px' src="https://mdn.mozillademos.org/files/8019/mdn-url-protocol@x2.png"></img>
<img width='480px' src="https://mdn.mozillademos.org/files/8021/mdn-url-protocol@x2.png"></img>
<img width='480px' src="https://mdn.mozillademos.org/files/8023/mdn-url-protocol@x2.png"></img>

https://developer.mozilla.org/ru/docs/Learn/Understanding_URLs

In [94]:
from urllib.parse import urlparse, parse_qsl

url = "http://www.example.com:80/path/to/myfile.html?key1=value1&key2=value2#SomewhereInTheDocument"
urlparse(url)

ParseResult(scheme='http', netloc='www.example.com:80', path='/path/to/myfile.html', params='', query='key1=value1&key2=value2', fragment='SomewhereInTheDocument')

In [95]:
dict(parse_qsl(urlparse(url).query))

{'key1': 'value1', 'key2': 'value2'}

# Типы HTTP запросов и ответов

<img width='500px' src="images/swagger-petstore-endpoint-methods.png"></img>

<img width='800px' src="images/responses.jpg"></img>

Табличка с типами ответов: [клац раз](https://www.steveschoger.com/status-code-poster/img/status-code.png) и [клац два](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP).

# Работа с JSON

In [96]:
import requests

[Документация](http://docs.python-requests.org/) по библиотеке requests.

In [97]:
import json

prepare token: https://developer.spotify.com/console/get-track/?id=3n3Ppam7vgaVa1iaRUc9Lp

In [98]:
with open('spotify-api-token.key', mode='r') as f_key:
    spotify_api_token = f_key.readline().strip()
    

class SpotifyAPI:    
    def __init__(self, token):
        self._token = token

    def get_track_info(self, track_id, market='US'):
        # посылаем get запрос, указываем аргументы и заголовки
        ret = requests.get("https://api.spotify.com/v1/tracks/{}".format(track_id),
                           params={'market': market,},
                           headers={'Authorization': f'Bearer {self._token}'})
        return ret


spotify = SpotifyAPI(spotify_api_token)
r = spotify.get_track_info('2Ygs64z9ywJGakuKU8tr6o')
r.status_code

400

In [99]:
r.headers

{'WWW-Authenticate': 'Bearer realm="spotify", error="invalid_request", error_description="Only valid bearer authentication supported"', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version', 'Access-Control-Allow-Methods': 'GET, POST, OPTIONS, PUT, DELETE, PATCH', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Max-Age': '604800', 'Content-Type': 'application/json', 'content-encoding': 'gzip', 'Date': 'Wed, 30 Oct 2019 16:32:55 GMT', 'Via': '1.1 google', 'Alt-Svc': 'clear', 'Transfer-Encoding': 'chunked'}

In [100]:
print(r.text)

{
  "error": {
    "status": 400,
    "message": "Only valid bearer authentication supported"
  }
}


In [101]:
answer = json.loads(r.text)
type(answer)

dict

In [102]:
answer = r.json()
type(answer)

dict

In [103]:
answer_cut = {
    'artist':   answer['artists'][0]['name'],
    'name':     answer['name'],
    'duration': answer['duration_ms'] / 1000.0,
    'href':     answer['href'],
    'id':       answer['id'],
    'preview':  answer['preview_url'],
}
answer_cut

KeyError: 'artists'

In [ ]:
filename = urlparse(answer_cut['preview']).path.split('/')[-1] + '.mp3'

with open(filename, 'wb') as f:
    data = requests.get(answer_cut['preview']).content
    f.write(data)
    
print(filename)

In [ ]:
import IPython.display as ipd

ipd.Audio(filename) 

In [ ]:
# компактно сериализуем json в строку

json.dumps(answer_cut, separators=(',', ':'))

In [ ]:
# pretty print для json

print(json.dumps(answer_cut, indent=4, sort_keys=True))

In [ ]:
with open('/tmp/answer_cut.json', mode='w') as f_json:
    json.dump(answer_cut, f_json)

In [ ]:
!cat /tmp/answer_cut.json

In [ ]:
%%bash

cat /tmp/answer_cut.json | python -m json.tool

In [ ]:
with open('/tmp/answer_cut.json', mode='r') as f_json:
    answer_cut = json.load(f_json)
answer_cut

## А можно ли как-то ускорить процесс обкачки?

<span style="color:blue;font-weight:bold">Вопрос:</span> какая разница между процессами и потоками (threads)?

<img width='600px' src="https://joearms.github.io/images/con_and_par.jpg"></img>

Глобальная блокировка интерпретатора ( global interpreter lock , **GIL** ) Python. Если два или более потока попытаются манипулировать одним и тем же объектом в одно и то же время, то неизбежно возникнут проблемы. Глобальная блокировка интерпретатора исправляет это. В любой момент времени действия может выполнять только один поток. Python автоматически переключается между потоками, когда в этом возникает необходимость.

GIL в Python реализован как обычный lock.

Скачивание данных через библиотеку requsets – I/O операция, поэтому большую часть времени процессор не выполняет никаких задач, потому что ждёт, пока данные придут по сети.

In [105]:
from tqdm import tqdm_notebook

In [106]:
%%time
from ipywidgets import IntProgress
n_post_first, n_post_final = 349000, 349200

r = [requests.get('https://habrahabr.ru/post/{}/'.format(post_id))
     for post_id in tqdm_notebook(range(n_post_first, n_post_final, 2))]

ImportError: IntProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [107]:
from multiprocessing.dummy import Pool as ThreadPool

In [108]:
%%time

n_post_first, n_post_final = 349000, 349200

def get_habr_post(post_id):
    return requests.get('https://habrahabr.ru/post/{}/'.format(post_id))

with ThreadPool(10) as pool:
    r = pool.map(get_habr_post, range(n_post_first, n_post_final, 2))
pool.join()   # same as wait pid

CPU times: user 8.01 s, sys: 342 ms, total: 8.35 s
Wall time: 23 s


In [109]:
from multiprocessing import Pool

In [110]:
%%time

n_post_first, n_post_final = 349000, 349200

def get_habr_post(post_id):
    return requests.get('https://habrahabr.ru/post/{}/'.format(post_id))

with Pool(processes=10) as pool:
    r = pool.map(get_habr_post, range(n_post_first, n_post_final, 2))
pool.join()   # same as wait pid

CPU times: user 146 ms, sys: 153 ms, total: 299 ms
Wall time: 22.3 s


# Парсинг HTML-страничек

<img width='800px' src="https://q-bit.biz/uploads/article/Tegs_1539003896.png"></img>

## robots.txt

1. Валидаторы:
    * https://webmaster.yandex.ru/tools/robotstxt/
    * https://technicalseo.com/seo-tools/robots-txt/
2. Все о robots.txt:
    * https://help.mail.ru/webmaster/indexing/robots.txt
    * https://yandex.ru/support/webmaster/controlling-robot/

## HTMLParser

https://www.kinopoisk.ru/film/garri-potter-i-taynaya-komnata-2002-688/

In [111]:
from html.parser import HTMLParser

class KinoPoiskParser(HTMLParser):
    """
    Парсер страниц КиноПоиска с фильмами.

    Извлекает следующую информацию:
        1. название;
        2. оригинальное название;
        3. страны производители;
        4. длительность;
        5. описание;
        6. рейтинг.
    """

    def __init__(self):
        super().__init__()
        self.film_info = dict()
        self.status = None
        
    def handle_starttag(self, tag, attrs):#setting handlers on tags
        attrs = dict(attrs)
        self.status = None
        
        # HTML Parser не умеет сплитить значения атрибутов по пробелу
        if 'class' in attrs:
            attrs['class'] = attrs['class'].split()
        else:
            attrs['class'] = []
        
        if tag == 'span':
            if 'moviename-title-wrapper' in attrs['class']:
                # 1. Название фильма
                self.status = 'title'
            elif attrs.get('itemprop', '') == 'alternativeHeadline':
                # 2. Оригинальное название фильма
                self.status = 'title-original'
            elif 'rating_ball' in attrs['class']:
                # 6. Рейтинг фильма
                self.status = 'rating'
        elif tag == 'div':
            if 'flag' in attrs['class']:
                # 3. Страны производители
                if 'countries' not in self.film_info:
                    self.film_info['countries'] = []
                
                country = attrs.get('title')
                if country:
                    self.film_info['countries'].append(country)
            elif attrs.get('itemprop') == 'description':
                # 5. Описание фильма
                self.status = 'description'
        elif tag == 'td' and 'time' in attrs['class'] and \
                attrs.get('id', '') == 'runtime':
            # 4. Длительность фильма
            self.status = 'time'
            
    def handle_data(self, data):
        data = data.strip()
        
        if not data:
            return
        
        if self.status is not None:
            if self.status == 'rating':
                # приводим рейтинг к float
                data = float(data)
            elif self.status == 'time':
                # форматируем дату
                data = re.search('\d+', data)
                data = int(data.group(0))
                data = "{:02}:{}".format(data // 60, data % 60)
            else:
                # в остальных случаях убираем лишние пробелы
                data = re.sub('\s+', ' ', data)
            self.film_info[self.status] = data 
            
    def close(self):
        # очищаем информацию о фильме, перед обработкой
        # новой страницы
        self.film_info = {}
        super().close()
        
parser = KinoPoiskParser()

In [123]:
film_url = 'https://www.kinopoisk.ru/film/garri-potter-i-taynaya-komnata-2002-688/'
html_1 = requests.get(film_url).text

In [113]:
# with open('data/file1.html') as f:
#     html_1 = f.read()

In [114]:
parser.feed(html_1)
parser.film_info

{}

In [115]:
film_url = 'https://www.kinopoisk.ru/film/258687/'
html_2 = requests.get(film_url).text

In [116]:
# with open('data/file2.html') as f:
#     html_2 = f.read()

In [117]:
parser.feed(html_2)
parser.film_info

{}

## lxml

<img width = '250px' src="images/lxml.png">

https://lxml.de/tutorial.html

In [118]:
from lxml import etree, html as lhtml
driver = webdriver.Firefox('/usr/lib/geckodriver')

NameError: name 'webdriver' is not defined

In [ ]:
tree = lhtml.fromstring(html_1)

Так как html и xml имеют древовидную структуру, до любого элемента всегда существет единственный путь, XPath.

https://www.w3schools.com/xml/xpath_intro.asp

```bash
pip install cssselect   # для подержки CSS селекторов
```

In [ ]:
film_info = {
    'title': tree.xpath('//span[@class="moviename-title-wrapper"]')[0].text,
    'title-original': tree.xpath('//span[@itemprop="alternativeHeadline"]')[0].text,
    'rating': float(tree.cssselect('.rating_ball')[0].text),   # поддержка CSS-селекторов
}

film_info

Распарсим длительность фильма.

In [ ]:
elem = tree.xpath('//td[@class="time"]')[0]#xpath is full path in tag tree
print(etree.tostring(elem, pretty_print=True, encoding='utf8', method='xml').decode('utf8').rstrip())
elem.text

In [ ]:
list(elem.itertext())

In [ ]:
tree.xpath('//td[@class="time"]/text()')

In [ ]:
film_info['time'] = tree.xpath('//td[@class="time"]/text()')[1].strip()
film_info

Распарсим страны производители фильма (по флагам).

In [ ]:
tree.xpath('//div[@class="movieFlags"]')

In [ ]:
tree.xpath('//div[contains(@class, "movieFlags")]')

In [ ]:
tree.xpath('//div[contains(@class, "movieFlags")]/div')

In [ ]:
info_cntr = tree.xpath('//div[@class="movieFlags movie-info__flags"]/div')
info_cntr = [e.attrib['title'] for e in info_cntr]
info_cntr

In [ ]:
info_cntr = tree.xpath('//div[@class="movieFlags movie-info__flags"]/div/attribute::title')

film_info['countries'] = info_cntr
film_info

Распарсим описание фильма.

In [ ]:
elem = tree.xpath('//div[@itemprop="description"]')[0]
print(etree.tostring(elem, pretty_print=True, encoding='utf8').decode('utf8').rstrip())
elem.text

In [ ]:
info_desc = tree.xpath('//div[@itemprop="description"]')[0]
info_desc = map(lambda s: re.sub('\s+', ' ', s), info_desc.itertext())
info_desc = '\n'.join(info_desc)
film_info['description'] = info_desc

film_info

## Beautiful Soup
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

<img width = '330px' src="images/bsoup.jpg">

In [ ]:
from bs4 import BeautifulSoup

In [125]:
film_html = html_1

In [129]:
soup = BeautifulSoup(film_html, 'html.parser')

In [127]:
soup = BeautifulSoup(film_html, 'lxml')

### Соберем основную информацию о фильме

In [133]:
film_info = {
    'title': soup.find('span', class_='moviename-title-wrapper').text,
    'title-original': soup.find('span', itemprop='alternativeHeadline').text,
    'countries': [e.attrs['title'] for e in soup.find_all('div', class_='flag')],
    'time': re.search('\d\d:\d\d', soup.find('td', class_='time').text).group(0),
    'rating': float(soup.select_one('.rating_ball').text)   # поддержка CSS-селекторов
}

film_info

AttributeError: 'NoneType' object has no attribute 'text'

In [134]:
desc = soup.find('div', itemprop='description')
desc

In [135]:
desc.text

AttributeError: 'NoneType' object has no attribute 'text'

In [136]:
for br in desc.find_all("br"):
    br.replace_with("\n")
desc

AttributeError: 'NoneType' object has no attribute 'find_all'

In [137]:
film_info['description'] = re.sub('\s+', ' ', desc.text)
film_info

AttributeError: 'NoneType' object has no attribute 'text'

### Извлечем таблицу

In [138]:
import pandas as pd

In [139]:
table = soup.find('table', class_='info')
rows = table.find_all('tr')

AttributeError: 'NoneType' object has no attribute 'find_all'

In [58]:
data = []

for row in rows:
    cols = map(lambda x: x.text, row.find_all('td'))
    data.append(cols)

data = pd.DataFrame(data)
data

,0,1
0,год,\n2002
1,страна,"\nВеликобритания, США, Германия"
2,слоган,«Зло вернулось в Хогвартс»
3,режиссер,Крис Коламбус
4,сценарий,"Стивен Кловз, Дж.К. Роулинг"
5,продюсер,"Дэвид Хейман, Майкл Барнатан, Дэвид Баррон,..."
6,оператор,Роджер Прэтт
7,композитор,Джон Уильямс
8,художник,"Стюарт Крэйг, Эндрю Эклэнд-Сноу, Марк Барто..."
9,монтаж,Питер Хонесс


In [59]:
data = []

for row in rows:
    cols = map(lambda x: x.text, row.find_all('td'))
    cols = map(lambda x: re.sub("\s+", ' ', x.strip()), cols)
    data.append(cols)

data = pd.DataFrame(data)
data

,0,1
0,год,2002
1,страна,"Великобритания, США, Германия"
2,слоган,«Зло вернулось в Хогвартс»
3,режиссер,Крис Коламбус
4,сценарий,"Стивен Кловз, Дж.К. Роулинг"
5,продюсер,"Дэвид Хейман, Майкл Барнатан, Дэвид Баррон, ..."
6,оператор,Роджер Прэтт
7,композитор,Джон Уильямс
8,художник,"Стюарт Крэйг, Эндрю Эклэнд-Сноу, Марк Бартолом..."
9,монтаж,Питер Хонесс


In [60]:
for row in rows:
    for script in row.find_all("script"):
        script.replace_with('')
        
data = []

for row in rows:
    cols = map(lambda x: x.text, row.find_all('td'))
    cols = map(lambda x: re.sub("\s+", ' ', x.strip()), cols)
    data.append(cols)

data = pd.DataFrame(data)
data

,0,1
0,год,2002
1,страна,"Великобритания, США, Германия"
2,слоган,«Зло вернулось в Хогвартс»
3,режиссер,Крис Коламбус
4,сценарий,"Стивен Кловз, Дж.К. Роулинг"
5,продюсер,"Дэвид Хейман, Майкл Барнатан, Дэвид Баррон, ..."
6,оператор,Роджер Прэтт
7,композитор,Джон Уильямс
8,художник,"Стюарт Крэйг, Эндрю Эклэнд-Сноу, Марк Бартолом..."
9,монтаж,Питер Хонесс


In [61]:
import pandas as pd

In [62]:
# Внимание! Из таблицы все равно придется убрать скрипты

df = pd.read_html(str(table))[0]
df

,0,1
0,год,2002
1,страна,"Великобритания, США, Германия"
2,слоган,«Зло вернулось в Хогвартс»
3,режиссер,Крис Коламбус
4,сценарий,"Стивен Кловз, Дж.К. Роулинг"
5,продюсер,"Дэвид Хейман, Майкл Барнатан, Дэвид Баррон, ..."
6,оператор,Роджер Прэтт
7,композитор,Джон Уильямс
8,художник,"Стюарт Крэйг, Эндрю Эклэнд-Сноу, Марк Бартолом..."
9,монтаж,Питер Хонесс


### Пройдемся по спискам актеров...

In [63]:
actors, actors_dub = soup.find('div', id='actorList').find_all('ul')

In [64]:
[row.text for row in actors.children if hasattr(row, 'text') and row.text != '...']#attribute of beutifulsoap

['Дэниэл Рэдклифф',
 'Руперт Гринт',
 'Эмма Уотсон',
 'Том Фелтон',
 'Кеннет Брана',
 'Бонни Райт',
 'Алан Рикман',
 'Ричард Харрис',
 'Мэгги Смит',
 'Робби Колтрейн']

In [65]:
[row.text for row in actors_dub.children if hasattr(row, 'text') and row.text != '...']

['Николай Быстров',
 'Ольга Сирина',
 'Лина Иванова',
 'Александр Скрывля',
 'Вадим Андреев']

### Обкачаем картинки

In [66]:
url = 'https://www.kinopoisk.ru/film/garri-potter-i-taynaya-komnata-2002-688/stills/'
film_photos_html = requests.get(url).content

In [67]:
soup = BeautifulSoup(film_photos_html, 'html.parser')

In [68]:
list(map(lambda s: s.attrs['src'], soup.find('table', class_='fotos').findAll("img")))[:10]

['https://st.kp.yandex.net/images/kadr/sm_2094224.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2005184.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2005183.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2005182.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2005181.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_1558456.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_1558455.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_1558454.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_1558453.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_1558452.jpg']

## Заголовки

https://www.dafont.com/

In [144]:
r = requests.get('https://www.dafont.com/')
len(r.text)

0

Библиотека для подстановки UserAgent: https://pypi.org/project/fake-useragent/

Что у меня за браузер: https://developers.whatismybrowser.com/useragents/parse/?analyse-my-user-agent=yes

In [169]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.1 Safari/605.1.15'}
r = requests.get('https://www.dafont.com/', headers=headers)#told the server(page) that we are browser
len(r.text)

18446

## Cookie

Cookie (куки) — это небольшие текстовые файлы, в которые браузер записывает данные с посещенных вами сайтов. Файлы cookie позволяют сайтам «запоминать» своих посетителей, например, чтобы каждый раз не переспрашивать их логин и пароль.

chrome://settings/cookies/detail?site=sphere.mail.ru (sessionid_gtp)

https://sphere.mail.ru/people/?q=BD-11

In [172]:
r = requests.get('https://sphere.mail.ru/people/', params={'q': 'BD-11'})
r.status_code

200

In [173]:
soup = BeautifulSoup(r.content, 'html.parser')

table = soup.find('div', class_='people-list')
table is not None

False

In [175]:
print(r.url)

https://sphere.mail.ru/pages/index/?next=/people/#auth


In [176]:
r = r.history[0]
r.status_code, r.is_redirect

(302, True)

In [177]:
print(r.url)

https://sphere.mail.ru/people/?q=BD-11


In [178]:
with open('sphere-cookies.json', mode='r') as f_cookies:#put our cookie here
    cookies = json.load(f_cookies)
    
r = requests.get('https://sphere.mail.ru/people/', params={'q': 'BD-11'}, cookies=cookies)
r.status_code

200

In [180]:
print(r.url)

https://sphere.mail.ru/pages/index/?next=/people/#auth


In [181]:
soup = BeautifulSoup(r.content, 'html.parser')

table = soup.find('div', class_='people-list')
table is not None

False

In [79]:
from urllib.parse import urlparse

In [80]:
rows = table.find('table', class_='table-users').find('tbody').find_all('tr')

users = [{'name': row.find('p', class_='realname').text,
          'nick': urlparse(row.find('p', class_='username').find('a').attrs['href']).\
              path.split('/')[2],
          'type': row.find('span', class_='user-group').text,
          'power': row.find('td', class_='cell-skill').text,
          'rating': row.find('td', class_='cell-rating').text.strip()}
         for row in rows]

df = pd.DataFrame(users)
df.head(10)

,name,nick,type,power,rating
0,Даниль Гайнанов,d.gajnanov,Студент,0.01,3.36
1,Фёдор Королёв,f.korolyov,Студент,0.06,3.36
2,Егор Коршунов,e.korshunov,Студент,0.00,2.94
3,Арсений Зотов,a.zotov,Студент,0.00,2.52
4,Александр Яковенко,a.yakovenko,Студент,0.00,2.10
5,Алексей Зеленцов,a.zelentsov,Студент,0.00,1.26
6,Алексей Яндутов,a.yandutov,Студент,0.00,1.26
7,Андрей Москаленко,and.moskalenko,Студент,0.00,1.26
8,Всеволод Денисов,vs.denisov,Студент,0.00,1.26
9,Вадим Вахрушев,v.vahrushev,Студент,0.00,1.26


## Proxy

In [182]:
def browser_stats_from_yandex(**params):
    r = requests.get('https://yandex.ru/internet/', **params)
    soup = BeautifulSoup(r.content, 'html.parser')

    params = {}
    for e in soup.find('ul', class_='general-info').find_all('li'):
        key = e.find('h3').text
        val = e.find('div', {'class': None})
        val = val.text if val else '–'
        params[key] = val
    return params

In [185]:
browser_stats_from_yandex()

{'IPv4-адрес': '188.44.42.216',
 'IPv6-адрес': '–',
 'Браузер': 'Неизвестный браузер',
 'Разрешение экрана': '–',
 'Регион': '–',
 'JavaScript отключен': 'В вашем браузере отсутствует или выключена поддержка JavaScript.'}

http://spys.one/proxys/DE/

In [187]:
proxies = { 'https': '167.71.33.215:3128' }
browser_stats_from_yandex(proxies=proxies)

KeyboardInterrupt: 

In [ ]:
browser_stats_from_yandex(proxies=proxies, headers=headers)

## Selenium

<img width = '500px' src="https://cdn-a.william-reed.com/var/wrbm_gb_food_pharma/storage/images/publications/food-beverage-nutrition/nutraingredients.com/news/research/selenium-may-protect-against-epileptic-seizures-mouse-study-suggests/7690019-1-eng-GB/Selenium-may-protect-against-epileptic-seizures-mouse-study-suggests_wrbm_large.jpg">

http://selenium-python.readthedocs.io/

https://kreisfahrer.gitbooks.io/selenium-webdriver/

In [188]:
url_format = "http://www.banki.ru/banks/map/{city}/#/!b1:{bank_ids}!s3:{bank_types}!s4:list!p1:{page}"

In [189]:
url_params = {
    'city': 'drugie/lyubertssyi',
    'bank_ids': 'all',
    'bank_types': 'all',
    'page': 1,
}

In [190]:
url_current = url_format.format(**url_params)
print(url_current)

http://www.banki.ru/banks/map/drugie/lyubertssyi/#/!b1:all!s3:all!s4:list!p1:1


In [191]:
r = requests.get(url_current).content

In [192]:
# find branches__list

soup = BeautifulSoup(r, "html.parser")
print(soup.find('div', {'class': 'branches__list'}))

<div class="branches__list branches__list--hidden" id="branches__list"></div>


<img width = '350px' src="images/javascript.jpg">

In [205]:
from selenium import webdriver

import html
from time import sleep



In [207]:
with webdriver.Firefox(executable_path = '/usr/lib/geckodriver') as driver:
    driver.get("https://mail.ru/")
    sleep(3)

In [214]:
def get_banks_info(driver, url, t_sleep=1):
    driver.get(url)
    sleep(t_sleep)
    
    result = []
    items = driver.find_elements_by_class_name("list__item")
    
    for item in items:
        classes = item.get_attribute('class').split()
        if 'list__item--header' in classes:
            continue
            
        bank_info = {
            'name': item.get_attribute("data-name"),
            'type': item.get_attribute("data-type"),
            'address': item.get_attribute("data-address"),
            'lat': item.get_attribute("data-latitude"),
            'lon': item.get_attribute("data-longitude"),
            'name_bank': item.find_element_by_class_name("item__name__bank").text,
            'access24h': 'круглосуточно' in
                item.find_element_by_class_name("item__schedule").text.lower(),
        }

        if bank_info['lat'] is not None:
            bank_info['lat'] = float(bank_info['lat'])

        if bank_info['lon'] is not None:
            bank_info['lon'] = float(bank_info['lon'])
            
        for f in ['address', 'name', 'name_bank']:
            if bank_info[f] is None:
                continue
            bank_info[f] = html.unescape(bank_info[f])

        result.append(bank_info)
        
    return result


with webdriver.Firefox(executable_path = '/usr/lib/geckodriver') as driver:
    result = get_banks_info(driver, url_current)

In [218]:
result = pd.DataFrame(result)
result.head()

,name,type,address,lat,lon,name_bank,access24h
0,Отделение,office,"Московская обл., г. Люберцы, ул. Красная, д. 4",55.681925,37.891755,Абсолют Банк,False
1,Банкомат,atm,"г. Люберцы, ул. Красная, д. 4",55.681923,37.891758,Абсолют Банк,False
2,Офис «Авангард-Экспресс» № 8835,office,"Московская обл., г. Люберцы, ул. 3-е Почтовое ...",55.682166,37.868119,Авангард,False
3,Офис «Авангард-Экспресс» № 8939,office,"Московская обл., г. Люберцы, ш. Новорязанское,...",55.669862,37.871511,Авангард,False
4,Офис «Авангард-Экспресс» № 8944,office,"Московская обл., г. Люберцы, ул. Побратимов, д. 7",55.692013,37.896669,Авангард,False


### А как можно эту задачу решать по-умному?

Идем в консольку браузера 😉

In [219]:
offset, limit = 0, 100

df_banks_coords = []

with tqdm_notebook() as pbar:
    while len(df_banks_coords) < 1000:
        params = {
            'jsonrpc': '2.0',
            'method': 'bankGeo/getObjectsByFilter',
            'params': {
                'offset': offset,
                'limit': limit,
                'type': ['office', 'branch', 'atm', 'cash', 'self_office'],
                'bank_id': ['36119', '325', '63520', '327', '2764'],
            },
            'id': ''
        }

        r = requests.post('https://www.banki.ru/api/', json=params)
        if r.status_code != 200:
            raise Exception('Request has failed.')

        r = r.json()
        data = r['result']['data']
        df_banks_coords.extend(data)

        if not pbar.total:
            pbar.total = r['result']['total']
        pbar.update(len(data))
            
        offset += len(data)
        
        if not len(df_banks_coords) < r['result']['total']:
            break
        
df_banks_coords = pd.DataFrame(df_banks_coords)

ImportError: IntProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [95]:
df_banks_coords.head()

,region_id,longitude,latitude,bank_id,icon_url,is_main,id,type,address,name,active,sort
0,12949,49.328675,56.238047,36119,/upload/iblock/8e2/akbars.gif,False,10979669,atm,"Республика Татарстан, с. Нижняя Береске, ул. С...",Банкомат,True,0
1,479,49.194306,55.724966,36119,/upload/iblock/8e2/akbars.gif,False,10976723,atm,"г. Казань, Оренбургский тракт, д. 170",Банкомат,True,0
2,13510,48.742856,55.747968,36119,/upload/iblock/8e2/akbars.gif,False,10976620,atm,"г. Иннополис, ул. Спортивная, д. 110",Банкомат,True,0
3,480,52.425787,55.763627,36119,/upload/iblock/8e2/akbars.gif,False,10978152,atm,"г. Набережные Челны, бул. Домостроителей, д. 4...",Банкомат,True,0
4,2484,53.228504,54.405133,36119,/upload/iblock/8e2/akbars.gif,False,10976185,atm,"г. Бавлы, ул. Энгельса, д. 55",Банкомат,True,0


### А что ещё умеет Selenium?

In [220]:
import random

def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)

with webdriver.Firefox(executable_path = '/usr/lib/geckodriver') as driver:
    driver.get("https://music.yandex.ru/artist/184100")
    random_sleep()
    
    # можно исполнять свой javascript код
    driver.execute_script("window.scrollTo(0, 250);")
    random_sleep(0)
    
    # можно использовать CSS-селекторы или XPath
    # лучше использовать Firefox, он лучше подсказывает пути
    # стоит использовать, если путь выглядит лаконичным
    # пример плохого CSS-селектора
    elem = driver.find_element_by_css_selector(
        'div.d-tabs__tab:nth-child(3) > a:nth-child(1)')
    
    # можно кликать по объектам
    # [ переходим на страничку с альбомами группы ]
    elem.click()
    random_sleep()
    
    # можно искать по классам
    albums = driver.find_element_by_class_name('page-artist__albums').\
        find_elements_by_class_name('album_selectable')
    album = albums[3]
    # [ прокручиваем до нужного альбома ]
    driver.execute_script("return arguments[0].scrollIntoView();", album)
    random_sleep()
    # [ переходим на страничку с выбранным альбомом ]
    album.find_element_by_class_name('d-link').click()
    random_sleep()
    
    driver.execute_script("window.scrollTo(0, 340);")
    random_sleep(0)
    
    # [ кликаем на play ]
    play = driver.find_element_by_class_name('button-play__type_album')
    play.click()
    
    # [ прокликиваем на 5 трек ]
    # пример хорошего CSS-селектора
    next_ = driver.find_element_by_css_selector('.d-icon_track-next')
    for i in range(4):
        next_.click()
        random_sleep(1, 1)
        
    # [ выводим информацию о треке ]
    # пример хорошего CSS-селектора
    elem = driver.find_element_by_css_selector('.track__name')
    artist = elem.find_element_by_class_name('track__artists').text
    title = elem.find_element_by_class_name('track__title').text
    
    tracks = driver.find_element_by_css_selector('.page-album__tracks').\
        find_elements_by_class_name('d-track')
    
    for track_i, track in enumerate(tracks, 1):
        if 'd-track_playing' in track.get_attribute("class").split(' '):
            break
    
    print("Now playing: {} - {}".format(artist, title))
    print("{}th track of {} tracks".format(track_i, len(tracks)))
    
    sleep(10)

WebDriverException: Message: Failed to decode response from marionette


И многое, многое другое... 😊

## Selenium — инструмент тестирования

[Katalon Recorder](https://www.katalon.com/) (Selenium IDE): [Chrome Extension](https://chrome.google.com/webstore/detail/katalon-recorder-selenium/ljdobmomdgdljniojadhoplhkpialdid), [Firefox Extension](https://addons.mozilla.org/en-US/firefox/addon/katalon-automation-record/)

chrome://extensions/

https://mail.ru/ --> "кто такой филантроп?"